In [1]:
%matplotlib inline
from utils import *
from sklearn.model_selection import train_test_split

In [2]:
train_a = pd.read_parquet('./data/A/train_targets.parquet')

X_train_estimated_a = pd.read_parquet('./data/A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('./data/A/X_train_observed.parquet')
X_test_estimated_a = pd.read_parquet('./data/A/X_test_estimated.parquet')

df = pd.concat([X_train_observed_a, X_train_estimated_a])

df = resample_to_hourly(df)
X_test_estimated_a = resample_to_hourly(X_test_estimated_a)

df = pd.merge(df, train_a, left_on='date_forecast', right_on='time', how='inner')
df = df.drop(columns=['snow_density:kgm3', 'snow_drift:idx', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_24h:cm', 'snow_depth:cm', 'snow_melt_10min:mm', 'snow_water:kgm2', 'elevation:m'])
# , 'snow_drift:idx', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_24h:cm', 'snow_depth:cm', 'snow_melt_10min:mm', 'snow_water:kgm2', 'wind_speed_w_1000hPa:ms', 'dew_point_2m:K', 'elevation:m', 'msl_pressure:hPa'
X_test_estimated_a = X_test_estimated_a.drop(columns=['snow_density:kgm3', 'snow_drift:idx', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_24h:cm', 'snow_depth:cm', 'snow_melt_10min:mm', 'snow_water:kgm2', 'elevation:m'])
# , 'snow_drift:idx', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_24h:cm', 'snow_depth:cm', 'snow_melt_10min:mm', 'snow_water:kgm2', 'wind_speed_w_1000hPa:ms', 'dew_point_2m:K', 'elevation:m', 'msl_pressure:hPa'

C:\Users\marku\Desktop\NTNU\ML\Testing\utils.py:99: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_hourly = df.resample('H').mean()
C:\Users\marku\Desktop\NTNU\ML\Testing\utils.py:99: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_hourly = df.resample('H').mean()


In [3]:
imputer = SimpleImputer(strategy='most_frequent')
X_test_estimated_a[['ceiling_height_agl:m', 'cloud_base_agl:m']] = imputer.fit_transform(X_test_estimated_a[['ceiling_height_agl:m', 'cloud_base_agl:m']])
df[['ceiling_height_agl:m', 'cloud_base_agl:m']] = imputer.fit_transform(df[['ceiling_height_agl:m', 'cloud_base_agl:m']])

In [4]:
df = is_estimated(df)
df = lag_features_by_one_hour(df, ['diffuse_rad_1h:J', 'direct_rad_1h:J', 'clear_sky_energy_1h:J'])
# df = feature_engineering_1(df)

X_test_estimated_a = lag_features_by_one_hour(X_test_estimated_a, ['diffuse_rad_1h:J', 'direct_rad_1h:J', 'clear_sky_energy_1h:J'], 'date_forecast')
X_test_estimated_a = is_estimated(X_test_estimated_a, 'date_forecast')
# X_test_estimated_a = feature_engineering_1(X_test_estimated_a)

In [5]:
train_end_date = '2022-10-21'
df['time'] = pd.to_datetime(df['time'])

train_df = df[df['time'] < train_end_date]
remaining_data = df[df['time'] > train_end_date]

train_data, validation_df = train_test_split(remaining_data, test_size=0.5, random_state=42)
train_df = pd.concat([train_df, train_data], ignore_index=True)

# Identifying the features and the target variable
X_train = train_df.drop(columns=['pv_measurement', 'time', 'date_forecast'])
y_train = train_df['pv_measurement']
X_val = validation_df.drop(columns=['pv_measurement', 'time', 'date_forecast'])
y_val = validation_df['pv_measurement']

In [6]:
# Combine training and validation data into a single dataset for AutoGluon
train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)

# Specify the name of the target variable
label = 'pv_measurement'

# Create a TabularPredictor object
predictor = TabularPredictor(label=label, eval_metric="mean_absolute_error").fit(train_data=train_data, tuning_data=val_data, presets='best_quality', num_gpus=1, num_stack_levels=0, use_bag_holdout=True)
# , num_gpus=1, num_stack_levels=0, use_bag_holdout=True

No path specified. Models will be saved in: "AutogluonModels\ag-20231029_150851\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231029_150851\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   937.83 GB / 2047.46 GB (45.8%)
Train Data Rows:    31863
Train Data Columns: 46
Tuning Data Rows:    2197
Tuning Data Columns: 46
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 649.75117, 1177.67732)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may spec

In [7]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                     model   score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L2  -85.369473       9.311466   999.878321                0.000000           0.220742            2       True         12
1        LightGBMXT_BAG_L1  -88.296414       9.105979   418.734988                9.105979         418.734988            1       True          3
2    NeuralNetTorch_BAG_L1  -89.087142       0.205487   580.922591                0.205487         580.922591            1       True         10
3     LightGBMLarge_BAG_L1  -93.884359      18.243578  1671.128786               18.243578        1671.128786            1       True         11
4          LightGBM_BAG_L1  -95.924798      11.123289   371.862375               11.123289         371.862375            1       True          4
5           XGBoost_BAG_L1  -99.592612       0.814488   134.000214  

C:\Users\marku\Desktop\Envs\MLenv\lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [8]:
y_pred = predictor.predict(X_test_estimated_a)
y_pred = y_pred.clip(lower=0)
y_pred = y_pred.reset_index(drop=True)
y_pred.index.name = 'id'

In [9]:
df = pd.DataFrame(y_pred)
df.to_csv('result_a.csv')